Combined Recommendation Engine

In [35]:
import pandas as pd
import pickle as pkl

In [81]:
df_subset = pd.read_csv('recipe_w_images.csv')
apriori_rules = pd.read_csv('recipe_mix.csv')

In [115]:
df_subset[df_subset['name'] == 'moist gingerbread cake']

,Unnamed: 0,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,Image
2,2,moist gingerbread cake,537543,55,2001201872,2018-11-16,"['60-minutes-or-less', 'time-to-make', 'course...","[1617.0, 104.0, 213.0, 8.0, 40.0, 203.0, 80.0]",8,"['preheat the oven to 350&deg', 'f grease a lo...",a slightly sticky loaf cake flavoured with gin...,"['unsalted butter', 'applesauce', 'egg', 'unsu...",10,True


In [37]:
#dummy data

user_rated_recipes = {5856:2, 5991:4, 2004:5}

In [66]:
recipe_rules = list(apriori_rules['Base Product'].unique())
print(recipe_rules)

[486261, 495124, 495275, 495577, 494631, 514965, 494435, 487073, 494972, 494609, 494784, 495271, 494671, 494303, 495152, 487492, 487593, 495967, 487551, 496573, 495134, 487039, 514423, 487669, 493958, 518145, 518069, 496591, 475041, 496730, 518068, 518151, 493963, 496552, 497295, 497130, 497382, 497236, 497261, 518143, 487568, 494953, 475780, 475819, 518229, 496767]


In [38]:
def recommendation_engine(user_rated_recipes):
    for k, v in user_rated_recipes.items(): 
        if not v < 3:
            print(k, '>', v)



In [39]:
recommendation_engine(user_rated_recipes)

5991 > 4
2004 > 5


In [40]:
with open('cosine-similarities.pkl', 'rb') as f:

    cosine_sim3 = pkl.load(f) # deserialize using load()

In [119]:
def get_recommendations(recipe_index, cosine_sim=cosine_sim3):

    remaining = 30 - min(30,len(apriori_rules[apriori_rules['Base Product'] == recipe_index]))


    indices = df_subset[['id', 'name']]

    idx = indices[indices['id'] == recipe_index].index[0]
    
    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))
    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # Get the scores of the 10 most similar movies
    top_similar = sim_scores[1:remaining+1]

    # Get the recipe indices and corresponding recipe similarity score
    recipe_indices = [i[0] for i in top_similar]
    recipe_similarity_scores = [i[1] for i in top_similar]

    recommendations_df = df_subset['name'].iloc[recipe_indices].to_frame().reset_index()
    recommendations_df['score'] = recipe_similarity_scores

# Return the top 10 most similar recipes
    return recommendations_df

In [120]:

get_recommendations(537543)  


,index,name,score
0,5856,gluten free gingerbread cake,0.526627
1,5991,gingerbread spice blend,0.425138
2,5983,tartine s soft glazed gingerbread,0.413226
3,2004,gingerbread smoothie,0.400864
4,2670,pumpkin gingerbread w caramel sauce,0.400677
5,4646,homemade brown sugar,0.336926
6,1281,ginger snaps,0.327445
7,5891,crunchy gingerbread cookies,0.324020
8,7079,applesauce apple butter,0.320176
9,7455,cardamom ginger crunch,0.317110


In [121]:
# pickle apriori rule csv
with open('./apriori-rules.pkl', 'wb') as f:
    pkl.dump(apriori_rules, f)

# add method to update apriori rules and resave csv + pickle